<a href="https://colab.research.google.com/github/sheon-j/news-wordcloud/blob/main/data/crawler/kid_chosun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib import request
from urllib.parse import quote
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd

In [80]:
# crawl href
def crawl_href(url):
  req = request.urlopen(url).read()
  soup = BeautifulSoup(req, 'html.parser')
  
  # get href
  target_news_list=[div.select_one('a').get('href') 
                    for div in soup.select('.con')]
  
  return target_news_list


# crawl data
def crawl_data(url):
  req = request.urlopen(url).read()
  soup = BeautifulSoup(req, 'html.parser')

  # get data from target news data
  json_data = {}
  # news_url
  json_data['news_url'] = url

  # news_source
  json_data['news_source'] = '어린이조선일보'

  # news_title
  news_title = soup.select_one('.page_title_subject').text
  json_data['news_title'] = news_title

  # news_date
  news_date = soup.select_one('.date').text
  news_date = re.sub('\s{2,}| : |[가-힣]', '', news_date)
  json_data['news_date'] = news_date

  # news_writer
  news_writer = soup.select_one('.author').text
  news_writer = re.sub('[^가-힣·= ]|\s{2,}', '', news_writer)
  json_data['news_writer'] = news_writer

  # news_subtitle
  if soup.select_one('#article h3'):
    news_subtitle = soup.select_one('#article h3').text
  else:
    news_subtitle = ''
  json_data['news_subtitle'] = news_subtitle

  # news_img
  if soup.select_one('#article img'):
    news_img = soup.select_one('#article img').get('src')
  else:
    news_img = ''
  json_data['news_img'] = news_img

  # news_article
  if soup.select_one('#article'):
    soup.dl.extract()
    news_article = soup.select_one('#article').text
    news_article = re.sub('[^가-힣a-zA-Z0-9. ]|\s{2,}', '', news_article).strip()
    news_article
  else:
    news_article = ''
  json_data['news_article'] = news_article

  return json_data

In [87]:
# target: 어린이조선일보
news_list = []
data = []

# 시사뉴스
kid_chosun_url = "http://kid.chosun.com"
news_list_url = "/list_kj.html?catid=111&pn="

In [97]:
# crawl href
for page in range(1, 25):
  news_list += crawl_href(kid_chosun_url + news_list_url + str(page))

In [100]:
# crawl news data
for news in news_list:
  data.append(crawl_data(kid_chosun_url + news))

In [103]:
crawl_href(kid_chosun_url + news_list_url + str(page))

['/site/data/html_dir/2022/03/01/2022030101725.html',
 '/site/data/html_dir/2022/03/01/2022030101725.html',
 '/site/data/html_dir/2022/03/01/2022030101723.html',
 '/site/data/html_dir/2022/03/01/2022030101720.html',
 '/site/data/html_dir/2022/03/01/2022030101714.html',
 '/site/data/html_dir/2022/03/01/2022030101660.html',
 '/site/data/html_dir/2022/03/01/2022030101739.html',
 '/site/data/html_dir/2022/03/01/2022030101734.html',
 '/site/data/html_dir/2022/03/01/2022030101731.html',
 '/site/data/html_dir/2022/02/28/2022022801787.html',
 '/site/data/html_dir/2022/02/28/2022022801767.html']

In [ ]:
# data preprocess
df = pd.DataFrame(data)

In [408]:
# save to csv
df.to_csv('adult_news_data.csv',mode = 'w', index=False)